In [1]:
# pull ACMI-linked Wikidata records to check multiple type declaration.

import numpy
import pandas
import pathlib
import pydash
import requests

def value_extract(row, column):

    ''' Extract dictionary values. '''
    
    return pydash.get(row[column], 'value')

def sparql_query(query, service):

    ''' Send sparql request, and formulate results into a dataframe. '''

    response = requests.get(service, params={'format': 'json', 'query': query}, timeout=120)
    results = pydash.get(response.json(), 'results.bindings')
    data_frame = pandas.DataFrame.from_dict(results)
    for column in data_frame.columns:
        data_frame[column] = data_frame.apply(value_extract, column=column, axis=1)
    
    return data_frame

wikidata_query = ''' 
    select distinct ?wikidata_id ?wikidata_typeLabel where { 
        ?wikidata_id wdt:P7003 ?acmi_id .
        ?wikidata_id wdt:P31 ?wikidata_type .
        service wikibase:label { bd:serviceParam wikibase:language "en". }
        } '''
wikidata_type = sparql_query(wikidata_query, 'https://query.wikidata.org/sparql')

wikidata_type['wikidata_id'] = wikidata_type['wikidata_id'].str.split('/').str[-1]
wikidata_type = wikidata_type.pivot_table(index=['wikidata_id'], aggfunc=lambda x: ','.join(x)).reset_index()
wikidata_type = wikidata_type.loc[wikidata_type.wikidata_typeLabel.str.contains(',', na=False)]

print(len(wikidata_type))
wikidata_type.sample(20)

147


,wikidata_id,wikidata_typeLabel
7837,Q632532,"television film,miniseries"
5636,Q371400,"business,talent agency"
7036,Q541667,"broadcaster,television production company"
9684,Q941279,"film,conflation"
683,Q1179100,"film,rockumentary"
5213,Q337078,"film,cult film"
3247,Q2208041,"film,3D film"
7569,Q59725808,"film,animated short film"
5592,Q3666999,"television film,animated film"
471,Q1140745,"film,sequel"
